# CHAPTER 38 - MANAGED ATTIBUTES

## WHY MANAGE ATTRIBUTES?

In [4]:
class Person:
    def getName(self):
        if not valid():
            raise TypeError('cannot featch name')
        else:
            return self.getName.transform()

    def setName(self, value):
        if not valid(value):
            raise TypeError('cannot change name')
        else:
            self.name = transform(value)

## PROPERTIES

In [1]:
class Person:
    def __init__(self, name):
        self._name = name
    def getName(self):
        print('fethc...')
        return self._name
    def setName(self, value):
        print('change...')
        self._name = value
    def delName(self):
        print('remove...')
        del self._name
    name = property(getName, setName, delName, "name property docs")

In [2]:
bob = Person('Bob Smith')

In [3]:
bob.name

fethc...


'Bob Smith'

In [6]:
bob.name = 'Robert Smith'

change...


In [7]:
bob.name

fethc...


'Robert Smith'

In [8]:
del bob.name

remove...


In [9]:
sue = Person('Sue Jones')

In [10]:
sue.name

fethc...


'Sue Jones'

In [11]:
Person.name.__doc__

'name property docs'

## COMPUTED ATTRIBUTES

In [15]:
class PropSquare:
    def __init__(self, start):
        self.value = start
    def getX(self):
        return self.value ** 2
    def setX(self, value):
        self.value = value
    X = property(getX, setX)

In [16]:
P = PropSquare(3)

In [17]:
Q = PropSquare(32)

In [19]:
P.X

9

In [21]:
Q.X

1024

## CODING PROPERTIES WITH DECORATORS

In [24]:
class Person:
    def __init__(self, name):
        self._name = name
    
    @property
    def name(self):
        "name property docs"
        print('fetch...')
        return self._name
    
    @name.setter
    def name(self, value):
        print('change...')
        self._name = value

    @name.deleter
    def name(self):
        print('remove...')
        del self._name

In [25]:
bob = Person('Bob Smith')

In [26]:
bob.name

fetch...


'Bob Smith'

In [27]:
del bob.name

remove...


In [29]:
sue = Person('Sue Jones')

In [30]:
sue.name

fetch...


'Sue Jones'

In [31]:
Person.name.__doc__

'name property docs'

## DESCRIPTORS

In [34]:
class Descriptor:
    "docstring goes here"
    def __get__(self, instance, owner): ...
    def __set__(self, instance, value): ...
    def __delete__(self, instance):...

In [35]:
class Descriptor:
    def __get__(self, instance, owner):
        print(self, instance, owner, sep='\n')

In [37]:
class Subject:
    attr = Descriptor()

In [38]:
X = Subject()

In [39]:
X.attr

<class '__main__.Subject'>


In [40]:
Subject.attr

None
<class '__main__.Subject'>


In [42]:
class D:
    def __get__(*args):
        print('get')

In [43]:
class C:
    a = D()

In [44]:
X = C()

In [45]:
X.a

get


In [46]:

C.a

get


In [49]:
X.a = 99

In [50]:
X.a

99

In [51]:
list(X.__dict__.keys())

['a']

In [52]:
Y = C()

In [53]:
Y.a

get


In [54]:
C.a

get


In [56]:
D.__dict__.keys()

dict_keys(['__module__', '__get__', '__dict__', '__weakref__', '__doc__'])

In [67]:
class D:
    def __get__(*args):
        print('get')
    def __set__(*args):
        raise AttributeError('cannot set')

In [68]:
class C:
    a = D()

In [69]:
X = C()

In [70]:
X.a

get


In [71]:
X.a = 99

AttributeError: cannot set

## A FIRST EXAMPLE

In [1]:
class Name:
    "name descriptor docs"
    def __get__(self, instance, owner):
        print('fetch...')
        return instance._name
    def __set__(self, instance, value):
        print('change...')
        instance._name = value
    def __delete__(self, instance):
        print('remove...')
        del instance._name

In [2]:
class Person:
    def __init__(self, name):
        self._name = name
    name = Name()

In [3]:
bob = Person('Bob Smith')

In [4]:
bob.name

fetch...


'Bob Smith'

In [5]:
bob.name = 'Robert Smith'

change...


In [7]:
bob.name

fetch...


'Robert Smith'

In [9]:
del bob.name

remove...


In [10]:
sue = Person('Sue Jones')

In [11]:
sue.name

fetch...


'Sue Jones'

In [12]:
Name.__doc__

'name descriptor docs'

In [13]:
class Person:
    def __init__(self, name):
        self._name = name

    class Name:
        "name descriptor docs"
        def __get__(self, instance, owner):
            print('fetch...')
            return instance._name
        def __set__(self, instance, value):
            print('change...')
            instance._name = value
        def __delete__(self, instance):
            print('remove...')
            del instance._name

    name = Name()

## COMPUTED ATTRIBUTES

In [15]:
class DescSquare:
    def __init__(self, start): # Each desc has own state
        self.value = start
    def __get__(self, instance, owner): # On attr fetch
        return self.value ** 2
    def __set__(self, instance, value): # On attr assign
        self.value = value # No delete or docs

In [16]:
class Client1:
    X = DescSquare(3) 

In [19]:
class Client2:
    X = DescSquare(32) 

In [20]:
c1 = Client1()

In [22]:
c2 = Client2()

In [25]:
c1.X

16

In [26]:
c2.X

1024

## USING STATE INFORMATION IN DESCRIPTORS

In [1]:
class DescState:
    def __init__(self, value):
        self.value = value
    def __get__(self, instance, owner):
        print('DescState get')
        return self.value * 10
    def __set__(self, instance, value):
        print('DescState set')
        self.value = value

In [2]:
class CalcAttrs:
    X = DescState(2)
    Y = 3
    def __init__(self):
        self.Z = 4

In [3]:
obj = CalcAttrs()

In [4]:
obj.X, obj.Y, obj.Z

DescState get


(20, 3, 4)

In [5]:
obj.X = 5

DescState set


In [6]:
CalcAttrs.Y = 6

In [7]:
obj.Z = 7

In [8]:
obj.X, obj.Y, obj.Z

DescState get


(50, 6, 7)

In [9]:
obj2 = CalcAttrs()

In [10]:

obj2.X, obj2.Y, obj2.Z

DescState get


(50, 6, 4)

In [6]:
class InsState:
    def __get__(self, instance, owner):
        print('InstState get')
        return instance.X * 10
    def __set__(self, instance, value):
        print('InstState set')
        instance._X = value

In [7]:
class CalcAttrs:
    X = InsState()
    Y = 3
    def __init__(self):
        self.X = 2
        self.Z = 4

In [8]:
obj = CalcAttrs()

InstState set


In [9]:
obj.X, obj.Y, obj.Z

In [1]:
class DescBoth:
    def __init__(self, data):
        self.data = data
    def __get__(self, instance, onew):
        return '%s, %s' % (self.data, instance.data)
    def __set__(self, instance, value):
        instance.data = value

In [3]:
class Client:
    def __init__(self, data):
        self.data = data
    managed = DescBoth('spam')

In [4]:
I = Client('eggs')

In [5]:
I.managed

'spam, eggs'

In [6]:
I.managed = 'SPAM'

In [7]:
I.managed

'spam, SPAM'

In [9]:
I.__dict__

{'data': 'SPAM'}

In [11]:
[x for x in dir(I) if not x.startswith('__')]

['data', 'managed']

In [12]:
getattr(I, 'data')

'SPAM'

In [13]:
getattr(I, 'managed')

'spam, SPAM'

In [14]:
for attr in (x for x in dir(I) if not x.startswith('__')):
    print('%s => %s' % (attr, getattr(I, attr)))

data => SPAM
managed => spam, SPAM


## HOW PROPERTIRES AND DESCRIPTORS RELATE 

In [106]:
class Property:
    def __init__(self, fget=None, fset=None, fdel=None, doc=None):
        self.fget = fget
        self.fset = fset
        self.fdel = fdel 
        self.__doc__ = doc

    def __get__(self, instance, instancetype=None):
        if instance is None:
            return self
        if self.fget is None:
            raise AttributeError("can't get attribute")
        return self.fget(instance)

    def __set__(self, instance, value):
        if self.fset is None:
            raise AttributeError("can't set attribute")
        self.fset(instance, value)

    def __delete__(self, instance):
        if self.fdel is None:
            raise AttributeError("can't delete attribute")
        self.fdel(instance)

In [107]:
class Person:
    def getName(self): print('getName...')
    def setName(self, value): print('setName...')
    name = Property(getName, setName)

In [108]:
x = Person()

In [109]:
x.name

getName...


In [110]:
x.name = 'Bob'

setName...


In [111]:
x.name

getName...


In [105]:
del x.name

AttributeError: can't delete attribute

 ## \_\_getattr__ AND \_\_getattribute__

In [114]:
class Catcher:
    def __getattr__(self, name):
        print(f'Get: {name}')
    def __setattr__(self, name, value):
        print(f'Set {name} {value}')

In [115]:
X = Catcher()

In [116]:
X.job

Get: job


In [117]:
X.pay

Get: pay


In [118]:
X.pay = 99

Set pay 99


In [120]:
class Wrapper:
    def __init__(self, objetct):
        self.wrapper = objetct
    def __getattr__(self, attrname):
        print('Trace:', attrname)
        return getattr(self.wrapper, attrname)

In [121]:
X = Wrapper([1, 2, 3])

In [122]:
X.append((4))

Trace: append


In [124]:
X.wrapper

[1, 2, 3, 4]

In [127]:
X.wrapper[1]

2

## FIRST EXAMPLE

In [130]:
class Person:
    def __init__(self, name):
        self._name = name

    def __getattr__(self, attr): 
        print('get: ' + attr)
        if attr == 'name': 
            return self._name
        else:
            raise AttributeError(attr)
    def __setattr__(self, attr, value):
        print('set: ' + attr)
        if attr == 'name':
            attr = '_name' 
        self.__dict__[attr] = value 

    def __delattr__(self, attr): 
        print('del: ' + attr)
        if attr == 'name':
            attr = '_name' 
        attr = '_name' 

In [131]:
bob = Person('Bob Smith') 

set: _name


In [133]:
bob.name

get: name


'Bob Smith'

In [134]:
bob.name = 'Robert Smith' 

set: name


In [135]:
bob.name

get: name


'Robert Smith'

In [136]:
del bob.name

del: name


## COMPUTED ATTRIBUTES

In [16]:
class AttrSquare:
    def __init__(self, start):
        self.value = start

    def __getattr__(self, attr):
        if attr == 'X':
            return self.value ** 2
        else:
            raise AttributeError(attr)

    def __setattr__(self, attr: str, value: int) -> None:
        if attr == 'X':
            attr = 'value'
        self.__dict__[attr] = value

In [17]:
A = AttrSquare(3)

In [18]:
B = AttrSquare(32)

In [19]:
A.X

9

In [21]:
A.X = 4

In [22]:
A.X

16

In [23]:
A.value

4

In [24]:
B.X

1024

## USING \_\_getattribute__

In [30]:
class AttrSquare:
    def __init__(self, start):
        self.value = start

    def __getattribute__(self, attr):
        if attr == 'X':
            return self.value ** 2
        else:
            return object.__getattribute__(self, attr)

    def __setattr__(self, attr, value):
        if attr == 'X':
            attr = 'value'
        object.__setattr__(self, attr, value)

## MANAGEMENT TECHNIQUES COMPARED

In [31]:
class Powers(object):
    def __init__(self, square, cube):
        self._square = square
        self._cube = cube

    def getSquare(self):
        return self._square ** 2
    
    def setSquare(self, value):
        self._square = value

    square = property(getSquare, setSquare)

    def getCube(self):
        return self._cube ** 3

    cube = property(getCube)

In [32]:
X = Powers(3, 4)

In [33]:
X.square

9

In [34]:
X.square = 5

In [35]:
X.square

25

In [36]:
class DescSquare(object):
    def __get__(self, instance, owner):
        return instance._square ** 2
    def __set__(self, instance, value):
        instance._square = value

class DescCube(object):
    def __get__(self, instance, owner):
        return instance._cube ** 3

class Powers(object):
    square = DescSquare()
    cube = DescCube()
    def __init__(self, square, cube):
        self._square = square
        self._cube = cube

In [37]:
X = Powers(3, 4)

In [38]:
X.square

9

In [39]:
X.square = 5

In [40]:
X.square

25

In [41]:
class Powers:
    def __init__(self, square, cube):
        self._square = square
        self._cube = cube

    def __getattr__(self, name):
        if name == 'square':
            return self._square ** 2
        elif name == 'cube':
            return self._cube ** 3
        else:
            raise TypeError('unknown attr:' + name)

    def __setattr__(self, name, value):
        if name == 'square':
            self.__dict__['_square'] = value
        else:
            self.__dict__[name] = value

In [46]:
class Powers():
    def __init__(self, square, cube):
        self._square = square
        self._cube = cube

    def __getattribute__(self, name):
        if name == 'square':
            return object.__getattribute__(self, '_square') ** 2
        elif name == 'cube':
            return object.__getattribute__(self, '_cube') ** 3
        else:
            return object.__getattribute__(self, name)

    def __setattr__(self, name, value):
        if name == 'square':
            object.__setattr__(self, '_square', value) # Or use __dict__
        else:
            object.__setattr__(self, name , value)

In [47]:
X = Powers(3, 4)

In [44]:
X._square

3

In [45]:
X._cube

4

## EXAMPLE: ATTRIBUTE VALIDATIONS

In [1]:
class CardHolder():
    acctlen = 8
    retireage = 59.5

    def __init__(self, acct, name, age, addr):
        self.acct = acct
        self.name = name
        self.age = age
        self.addr = addr

    def getName(self):
        return self.__name

    def setName(self, value):
        value = value.lower().replace(' ', '_')
        self.__name = value

    name = property(getName, setName)

    def getAge(self):
        return self.__age
    def setAge(self, value):
        if value < 0 or value > 150:
            raise ValueError('invalid age')
        else:
            self.__age = value

    age = property(getAge, setAge)

    def getAcct(self):
        return self.__acct[:-3] + '***'
    
    def setAcct(self, value):
        value = value.replace('-', '')
        if len(value) != self.acctlen:
            raise TypeError('invald acct number')
        else:
            self.__acct = value

    acct = property(getAcct, setAcct)

    def remainGet(self):
        return self.retireage - self.age

    remain = property(remainGet)

In [2]:
CardHolder('1234-5678', 'Bob Smith', 40, '123 main st')

In [3]:
def printholder(who):
    print(who.acct, who.name, who.age, who.remain, who.addr, sep=' / ')

bob = CardHolder('1234-5678', 'Bob Smith', 40, '123 main st')
printholder(bob)

bob.name = 'Bob Q. Smith'
bob.age = 50
bob.acct = '23-45-67-89'
printholder(bob)

sue = CardHolder('5678-12-34', 'Sue Jones', 35, '124 main st')
printholder(sue)

try:
    sue.age = 200
except:
    print('Bad age for Sue')

try:
    sue.remain = 5
except:
    print("Can't set sue.remain")
try:
    sue.acct = '1234567'
except:
    print('Bad acct for Sue')

12345*** / bob_smith / 40 / 19.5 / 123 main st
23456*** / bob_q._smith / 50 / 9.5 / 123 main st
56781*** / sue_jones / 35 / 24.5 / 124 main st
Bad age for Sue
Can't set sue.remain
Bad acct for Sue


In [9]:
bob.__dict__

{'_CardHolder__acct': '23456789',
 '_CardHolder__name': 'bob_q._smith',
 '_CardHolder__age': 50,
 'addr': '123 main st'}

In [11]:
sue.retireage

59.5

In [12]:
bob.retireage

59.5

## USING DESCRIPTORS TO VALIDADE

In [14]:
class CardHolder():
    acctlen = 8
    retireage = 59.5

    def __init__(self, acct, name, age, addr):
        self.acct = acct
        self.name = name
        self.age = age
        self.addr = addr

    class Name():
        def __get__(self, instance, owner):
            return self.__name
        def __set__(self, instance, value):
            value = value.lower().replace(' ', '_')
            instance.__name = value

    name = Name() 

    class Age():
        def __get__(self, instance, owner):
            return instance.__age
        def __set__(self, instance, value):
            if value < 0 or value > 150:
                raise ValueError('invalid age')
            else:
                instance.__age = value

    age = Age()

    class Acct():
        def __get__(self, instance, owner):
            return self.acct[:-3] + '***'

        def __set__(self, instance, value):
            value = value.replace('-', '')
            if len(value) != instance.acctlen:
                raise TypeError('invald acct number')
            else:
                instance.__acct = value

    acct = Acct()   

    class Remain():
        def __get__(self, instance, owner):
            return instance.retireage - instance.age
        
        def __set__(self, instance, value):
            raise TypeError('cannot set remain')

    remain = Remain()

## USING \_\_getattr__ TO VALIDADE

In [15]:
class CardHolder:
    acctlen = 8
    retireage = 59.5

    def __init__(self, acct, name, age, addr):
        self.acct = acct
        self.name = name
        self.age = age
        self.addr = addr

    def __getattr__(self, name):
        if name == 'acct':
            return self._acct[:-3] + '***'
        elif name == 'remain':
            return self.retireage - self.age
        else:
            raise AttributeError(name)

    def __setattr__(self, name, value):
        if name == 'name':
            value = value.lower().replace(' ', '_')
        elif name == 'age':
            if value < 0 or value > 150:
                raise ValueError('invalid age')
        elif name == 'acct':
            name = '_acct'
            value = value.replace('-', '')
            if len(value) != self.acctlen:
                raise TypeError('invald acct number')
        elif name == 'remain':
            raise TypeError('cannot set remain')
        
        self.__dict__[name] = value   

## USING \_\_getattribute__ TO VALIDADE

In [ ]:
class CardHolder():
    acctlen = 8
    retireage = 59.5

    def __init__(self, acct, name, age, addr):
        self.acct = acct
        self.name = name
        self.age = age
        self.addr = addr

    def __getattribute__(self, name):
        superget = object.__getattribute__
        if name == 'acct':
            return superget(self, 'acct')[:-3] + '***'
        elif name == 'remain':
            return superget(self, 'retireage') - superget(self, 'age')
        else:
            return superget(self, name)
        
    def __setattr__(self, name, value):
        if name == 'name':
            value = value.lower().replace(' ', '_')
        elif name == 'age':
            if value < 0 or value > 150:
                raise ValueError('invalid age')
        elif name == 'acct':
            value = value.replace('-', '')
            if len(value) != self.acctlen:
                raise TypeError('invald acct number')
            raise TypeError('cannot set remain')
        self.__dict__[name] = value